This notebook inspects the input shape for LayoutLM's masked language modeling.

For this task, LayoutLM expects these inputs:

1. a sequence of strings where each string is a word (or token)
2. the corresponding bounding box for each string (i.e., 4 numbers representing the diagonal
   coordinates). The coordinates are normalized to \[0, 1000\].

In [20]:
%matplotlib inline
%load_ext autoreload
%autoreload 3
%config IPythonBackend.figure_format = 'retina'

import os
from dataclasses import asdict

from project_config import DataPaths, S3DataPaths
from transformers import AutoTokenizer

from my_code.data import mlm
from my_code.train import get_model

try:
    import rich

    rich.reconfigure(force_terminal=True, force_jupyter=False)
    rich.pretty.install()
    print = rich.get_console().out
    rprint = rich.get_console().print
except:
    pass

def mydir(o):
    return [s for s in dir(o) if not s.startswith('_')]

datapaths_local = DataPaths()
datapaths_s3 = S3DataPaths()
display(asdict(datapaths_local), asdict(datapaths_s3))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

{
    'data_prefix': PosixPath('data'),
    'imgs': PosixPath('data/imgs-clean'),
    'textracted': PosixPath('data/textracted'),
    'annotations': PosixPath('data/annotations')
}

{
    'data_prefix': S3Path('s3://sagemaker-ap-southeast-1-111122223333/textract-transformers/data'),
    'imgs': PosixPath('data/imgs-clean'),
    'textracted': PosixPath('data/textracted'),
    'annotations': PosixPath('data/annotations'),
    'bucket_name': S3Path('s3://sagemaker-ap-southeast-1-111122223333'),
    'bucket_prefix': 'textract-transformers'
}


In [2]:
# Let's figure out the tokenizer to use, by looking at the mlm-pretraining job log.
rprint(
    "ModelArguments(cache_dir='/tmp/transformers/cache', config_name=None, model_name_or_path='microsoft/layoutlm-base-uncased', model_revision='main', tokenizer_name=None, use_auth_token=False)"
)
print()
rprint(
    "DataTrainingArguments(annotation_attr='labels', max_seq_length=512, max_train_samples=None, task_name='mlm', textract='/opt/ml/input/data/textract', textract_prefix='textract-transformers/data/textracted', train='/opt/ml/input/data/train', validation='/opt/ml/input/data/validation', num_labels=2, mlm_probability=0.15)"
)

ModelArguments(cache_dir='/tmp/transformers/cache', config_name=None, 
model_name_or_path='microsoft/layoutlm-base-uncased', model_revision='main', 
tokenizer_name=None, use_auth_token=False)

DataTrainingArguments(annotation_attr='labels', max_seq_length=512, 
max_train_samples=None, task_name='mlm', textract='/opt/ml/input/data/textract',
textract_prefix='textract-transformers/data/textracted', 
train='/opt/ml/input/data/train', validation='/opt/ml/input/data/validation', 
num_labels=2, mlm_probability=0.15)


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # For notebook only.

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    cache_dir=datapaths_local.data_prefix / "transformers/cache",
    use_fast=True,
    revision="main",
    use_auth_token=False,
)

ds_mlm = mlm.TextractLayoutLMDatasetForLM(
    textract_path=str(datapaths_local.textracted),
    tokenizer=tokenizer,
)

[WARNING|tokenization_utils_base.py:3325] 2022-01-20 02:11:17,493 >> Token indices sequence length is longer than the specified maximum sequence length for this model (924 > 512). Running this sequence through the model will result in indexing errors


In [21]:
mydir(ds_mlm)


[
    'dataset_inputs',
    'example_index',
    'functions',
    'manifest_file_path',
    'max_content_seq_len',
    'max_seq_len',
    'parse_textract_file',
    'register_datapipe_as_function',
    'register_function',
    'splitter',
    'textract_path',
    'textract_prefix',
    'textract_s3uri_to_file_path',
    'tokenizer'
]


In [13]:
display(
    ds_mlm.textract_path,
    "<NONE>" if ds_mlm.manifest_file_path is None else ds_mlm.manifest_file_path,
    ds_mlm.textract_prefix,
    len(ds_mlm.example_index),
)

'data/textracted/'
'<NONE>'
''
2544


In [15]:
first_doc = ds_mlm.example_index[0]
print(first_doc)

TextractLayoutLMExampleForLM(word_boxes_normalized=array([[568.13204288,  49.72891137, 701.71193779,  63.92122339],
       [544.09110546,  69.7896257 , 630.71605563,  84.08804704],
       [635.99562645,  70.1417774 , 700.49893856,  84.08466354],
       ...,
       [604.19762135, 617.5339222 , 613.47612366, 625.69778133],
       [617.8407073 , 615.1227355 , 679.96006832, 625.22857077],
       [685.43183804, 615.04977942, 759.9587664 , 625.48963912]]), word_texts=['CONSUMER', 'CREDIT', 'CARD', 'AGREEMENT', 'VISA', 'AND', 'DISCLOSURE', 'VISA', 'PLATINUM/VISA', 'PLATINUM', 'REWARDS/VISA', 'PLATINUM', 'SECURED', 'This', 'Consumer', 'Credit', 'Card', 'Agreement', 'and', 'Disclosure', 'together', 'with', 'the', 'Account', 'Opening', 'Disclosure', 'and', 'any', 'other', 'Account', 'opening', 'documents', 'or', 'any', 'subsequent', 'documents', 'provided', 'to', 'You', 'related', 'to', 'this', 'Account', '(hereinafter', 'collectively', 'referred', 'to', 'as', '"Agreement")', 'govern', 'the', 't

In [30]:
# Based on GITROOT/notebooks/src/my_code/data/mlm.py::TextractLayoutLMDatasetForLM.__init__()
self = ds_mlm  # Alias, to minimize changes to the copy-pasted code :P

from my_code.data.geometry import layoutlm_boxes_from_trp_blocks
from my_code.data.mlm import TextractLayoutLMExampleForLM

record = next(self.dataset_inputs())
textract_file_path = record["textract-ref"]
page_num = record.get("page-num")
doc = self.parse_textract_file(textract_file_path)
for page in doc.pages[
    # Filter to target page if provided, else process all pages:
    slice(None)
    if page_num is None
    else slice(page_num - 1, page_num)
]:
    words = [word for line in page.lines for word in line.words]
    word_boxes = layoutlm_boxes_from_trp_blocks(words)
    word_texts = [word.text for word in words]
    for start_word, end_word in self.splitter.split(
        word_texts,
        tokenizer,
        self.max_content_seq_len,
    ):
        self.example_index.append(
            TextractLayoutLMExampleForLM(
                word_boxes_normalized=word_boxes[start_word:end_word, :],
                word_texts=word_texts[start_word:end_word],
            )
        )

del self
rprint("Amazing, it worked :joy:")

Amazing, it worked 😂


In [51]:
rprint("Let's start inspecting the temporary variables :eyes:")
print(word_texts, word_boxes, ds_mlm.example_index[-1].word_boxes_normalized, sep='\n\n')
display(
    ds_mlm.max_content_seq_len,
    word_boxes.shape,
    len(word_texts),
)

Let's start inspecting the temporary variables 👀
['Your', 'Billing', 'Rights:', 'Keep', 'this', 'Document', 'for', 'Future', 'Use', 'This', 'notice', 'tells', 'You', 'about', 'Your', 'rights', 'and', 'Our', 'responsibilities', 'under', 'the', 'Fair', 'Credit', 'Billing', 'Act.', 'What', 'To', 'Do', 'If', 'You', 'Find', 'A', 'Mistake', 'On', 'Your', 'Statement', 'If', 'You', 'think', 'there', 'is', 'an', 'error', 'on', 'Your', 'statement,', 'write', 'to', 'Us', 'at', 'the', 'address', 'listed', 'on', 'Your', 'statement.', 'In', 'Your', 'letter,', 'give', 'Us', 'the', 'following', 'information:', '-', 'Account', 'information:', 'Your', 'name', 'and', 'Account', 'number.', '-', 'Dollar', 'amount:', 'The', 'dollar', 'amount', 'of', 'the', 'suspected', 'error.', '-', 'Description', 'of', 'problem:', 'If', 'You', 'think', 'there', 'is', 'an', 'error', 'on', 'Your', 'bill,', 'describe', 'what', 'You', 'believe', 'is', 'wrong', 'and', 'why', 'You', 'believe', 'it', 'is', 'a', 'mistake.', 'You'